Sample video for object detection: https://www.youtube.com/playlist?list=PLcQZGj9lFR7y5WikozDSrdk6UCtAnM9mB

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
import argparse
import os
from pathlib import Path
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import random
%matplotlib inline

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
# Activate google drive for colab dataset
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

In [ ]:
video_path = '/content/vid2.mp4'

In [ ]:
model = YOLO("yolov8n.pt")

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the video width, height, and FPS to create the output video file
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_path = '/content/predicted_video2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)

    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0].tolist())
        conf = result.conf[0].item()
        cls = result.cls[0].item()

        # Draw bounding box and label
        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with the detection boxes to the output video file
    out.write(frame)

# Release the video capture and writer objects
cap.release()
out.release()

# Optional: Display the path to the saved video
print(f"Predicted video saved at: {output_path}")